# Java Unit Tests

In [0]:
// Custom Imports
List <String> addedJars = %jars *.jar
//*/
%maven junit:junit:4.12
%maven org.apache.commons:commons-io:1.3.2
%maven commons-io:commons-io:2.7

// CryptoGuard imports
import static org.junit.Assert.assertFalse;
import static org.junit.Assert.assertTrue;
import static org.junit.Assert.assertEquals;
import static org.junit.Assert.assertNull;
import static org.junit.Assert.assertNotNull;
import static util.Utils.makeArg;
import frontEnd.Interface.outputRouting.ExceptionHandler;
import frontEnd.Interface.ArgumentsCheck;
import frontEnd.MessagingSystem.routing.Listing;
import frontEnd.MessagingSystem.routing.EnvironmentInformation;
import frontEnd.MessagingSystem.routing.structure.Default.Report;
import frontEnd.MessagingSystem.routing.structure.Scarf.AnalyzerReport;
import frontEnd.MessagingSystem.routing.structure.Scarf.BugInstance;
import frontEnd.Interface.EntryPoint;
import rule.engine.EngineType;
import frontEnd.argsIdentifier;
import java.io.File;
import java.nio.charset.StandardCharsets;
import java.nio.file.Files;
import java.nio.file.Paths;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;
import org.junit.After;
import org.junit.Before;
import org.junit.Test;
import org.junit.runner.RunWith;
import soot.G;
import util.Utils;
import org.apache.commons.io.FileUtils;
import java.util.Scanner;
import org.apache.commons.io.filefilter.WildcardFileFilter;

In [0]:
// region String Statics
public static final Boolean isLinux = !System.getProperty("os.name").contains("Windows");
public static final Boolean activate = true;

public static final String basePath = System.getProperty("user.dir").replace("/Notebook", "");
public static final String scarfArgs = Utils.osPathJoin(basePath, "src", "main", "resources", "Scarf", "sample.properties");
public static final String testRec = Utils.osPathJoin(basePath, "samples");
public static final String testPath = Utils.osPathJoin(System.getProperty("user.dir"), "tmp");

FileUtils.deleteDirectory(new File (testPath));
new File(testPath).mkdirs();

public static final String verySimple_Gradle = Utils.osPathJoin(testRec, "VerySimple_Gradle");
public static final String crypto_Example = Utils.osPathJoin(testRec, "Crypto_Example");
public static final String passwordUtils_Example = Utils.osPathJoin(testRec, "PasswordUtils_Example");
public static final String symCrypto_Example = Utils.osPathJoin(testRec, "SymCrypto_Example");
public static final String symCrypto_Package_Example = Utils.osPathJoin(testRec, "SymCrypto_Package_Example");
public static final String symCrypto_Multiple_Example = Utils.osPathJoin(testRec, "SymCrypto_Multi_Example");

In [0]:
public static final String verySimple_Gradle_File = Utils.osPathJoin(verySimple_Gradle, "src", "main", "java", "very.java");
public static final String tempFileOutJson_File = Utils.osPathJoin(testPath, "very-simple_File.json");
public static final String crypto_Example_File = Utils.osPathJoin(crypto_Example, "src", "main", "java", "Crypto.java");
public static final String crypto_Example_Json_File = Utils.osPathJoin(testPath, "Crypto_Example_File.json");
public static final String passwordUtils_Example_File = Utils.osPathJoin(passwordUtils_Example, "src", "main", "java", "PasswordUtils.java");
public static final String passwordUtils_Example_Json_File = Utils.osPathJoin(testPath, "PasswordUtils_Example_File.json");
public static final String symCrypto_Example_File = Utils.osPathJoin(symCrypto_Example, "src", "main", "java", "SymCrypto.java");
public static final String symCrypto_Example_Json_File = Utils.osPathJoin(testPath, "SymCrypto_Example_File.json");
public static final String symCrypto_Example_Package_File = Utils.osPathJoin(symCrypto_Package_Example, "src", "main", "java", "tester", "SymCrypto.java");
public static final String symCrypto_Example_Package_Json_File = Utils.osPathJoin(testPath, "SymCrypto_Package_Example_File.json");
public static final String symCrypto_Multiple_Example_File_1 = Utils.osPathJoin(symCrypto_Multiple_Example, "src", "main", "java", "SymCrypto.java");
public static final String symCrypto_Multiple_Example_File_2 = Utils.osPathJoin(symCrypto_Multiple_Example, "src", "main", "java", "PassEncryptor.java");
public static final String symCrypto_Multiple_Files = Utils.osPathJoin(testPath, "SymCrypto_Example_Files.json");

In [0]:
// Utililties
String cur_user = System.getProperty("user.name");
String java_home = "/home/" + cur_user + "/.sdkman/candidates/java/8*.0.25*";
String java7_home = "/home/" + cur_user + "/.sdkman/candidates/java/7*";
String android_home = "/home/" + cur_user + "/.sdkman/candidates/android/current";

//Retrieving a specific path from a wild card based list
public static String getFileFromWildCard(String path) {
  String[] split = path.split("/");
  StringBuilder build = new StringBuilder();
  for (int itr = 0;itr < split.length - 1;itr ++)
      build.append(split[itr]).append("/");

  String[] files = new File(build.toString()).list(new WildcardFileFilter(split[split.length-1]));

  if (files.length > 0)
      return build.toString() + files[files.length - 1];
  else
      return null;
}

java_home = getFileFromWildCard(java_home);
java7_home = getFileFromWildCard(java7_home);

public static void print(String file) {
  try {
    Scanner reader = new Scanner(new File(file));

    while (reader.hasNextLine()) System.out.println(reader.nextLine());
  } catch (FileNotFoundException e) {
  }
}

In [0]:
public void main_TestableFile_VerySimple() {
  soot.G.v().reset();

  String source = verySimple_Gradle_File;
  String fileOut = tempFileOutJson_File;

  new File(fileOut).delete();
  if (activate && isLinux) {
    String args =
        makeArg(argsIdentifier.FORMAT, EngineType.JAVAFILES)
            + makeArg(argsIdentifier.FORMATOUT, Listing.Default)
            + makeArg(argsIdentifier.SOURCE, source)
            + makeArg(argsIdentifier.NOEXIT)
            + makeArg(argsIdentifier.PRETTY)
            + makeArg(argsIdentifier.VERYVERBOSE)
            + makeArg(argsIdentifier.JAVA, java7_home)
            + makeArg(argsIdentifier.OUT, fileOut);

    try {
      EntryPoint.main(args.split(" "));
      Report report = Report.deserialize(new File(fileOut));

      assertFalse(report.getIssues().isEmpty());
      assertTrue(
          report.getIssues().stream()
              .anyMatch(
                  bugInstance -> {
                    try {
                      return bugInstance
                          .getFullPath()
                          .contains(Utils.retrieveFullyQualifiedName(source));
                    } catch (ExceptionHandler exceptionHandler) {
                      exceptionHandler.printStackTrace();
                      return false;
                    }
                  }));
      print(fileOut);
    } catch (Exception e) {
      e.printStackTrace();
      assertNull(e);
    }
  }
}

In [0]:
main_TestableFile_VerySimple()

In [0]:
public void main_Crypto_Example_File() {
  soot.G.v().reset();

  String source = crypto_Example_File;
  String fileOut = crypto_Example_Json_File;

  new File(fileOut).delete();
  if (activate && isLinux) {
    String args =
        makeArg(argsIdentifier.FORMAT, EngineType.JAVAFILES)
            + makeArg(argsIdentifier.FORMATOUT, Listing.Default)
            + makeArg(argsIdentifier.SOURCE, source)
            + makeArg(argsIdentifier.NOEXIT)
            + makeArg(argsIdentifier.ANDROID, "/InvalidPath/")
            + makeArg(argsIdentifier.PRETTY)
            + makeArg(argsIdentifier.JAVA, java7_home)
            + makeArg(argsIdentifier.VERYVERBOSE)
            + makeArg(argsIdentifier.OUT, fileOut);

    try {
      EntryPoint.main(args.split(" "));
      Report report = Report.deserialize(new File(fileOut));

      assertFalse(report.getIssues().isEmpty());
      assertTrue(
          report.getIssues().stream()
              .anyMatch(
                  bugInstance -> {
                    try {
                      return bugInstance
                          .getFullPath()
                          .contains(Utils.retrieveFullyQualifiedName(source));
                    } catch (ExceptionHandler exceptionHandler) {
                      exceptionHandler.printStackTrace();
                      return false;
                    }
                  }));
      print(fileOut);
    } catch (Exception e) {
      e.printStackTrace();
      assertNull(e);
    }
  }
}

In [0]:
main_Crypto_Example_File()

In [0]:
public void main_PasswordUtils_Example_File() {
  soot.G.v().reset();

  String source = passwordUtils_Example_File;
  String fileOut = passwordUtils_Example_Json_File;

  new File(fileOut).delete();
  if (activate && isLinux) {
    String args =
        makeArg(argsIdentifier.FORMAT, EngineType.JAVAFILES)
            + makeArg(argsIdentifier.FORMATOUT, Listing.Default)
            + makeArg(argsIdentifier.SOURCE, source)
            + makeArg(argsIdentifier.NOEXIT)
            + makeArg(argsIdentifier.ANDROID, "/InvalidPath/")
            + makeArg(argsIdentifier.PRETTY)
            + makeArg(argsIdentifier.JAVA, java7_home)
            + makeArg(argsIdentifier.VERYVERBOSE)
            + makeArg(argsIdentifier.OUT, fileOut);

    try {
      EntryPoint.main(args.split(" "));
      Report report = Report.deserialize(new File(fileOut));

      assertFalse(report.getIssues().isEmpty());
      assertTrue(
          report.getIssues().stream()
              .anyMatch(
                  bugInstance -> {
                    try {
                      return bugInstance
                          .getFullPath()
                          .contains(Utils.retrieveFullyQualifiedName(source));
                    } catch (ExceptionHandler exceptionHandler) {
                      exceptionHandler.printStackTrace();
                      return false;
                    }
                  }));
      print(fileOut);
    } catch (Exception e) {
      e.printStackTrace();
      assertNull(e);
    }
  }
}

In [0]:
main_PasswordUtils_Example_File()

In [0]:
public void main_SymCrypto_Example_File() {
  soot.G.v().reset();

  String source = symCrypto_Example_File;
  String fileOut = symCrypto_Example_Json_File;

  new File(fileOut).delete();
  if (activate && isLinux) {
    String args =
        makeArg(argsIdentifier.FORMAT, EngineType.JAVAFILES)
            + makeArg(argsIdentifier.FORMATOUT, Listing.Default)
            + makeArg(argsIdentifier.SOURCE, source)
            + makeArg(argsIdentifier.NOEXIT)
            + makeArg(argsIdentifier.PRETTY)
            + makeArg(argsIdentifier.JAVA, java7_home)
            + makeArg(argsIdentifier.VERYVERBOSE)
            + makeArg(argsIdentifier.OUT, fileOut);

    try {
      EntryPoint.main(args.split(" "));
      Report report = Report.deserialize(new File(fileOut));

      assertFalse(report.getIssues().isEmpty());
      assertTrue(
          report.getIssues().stream()
              .anyMatch(
                  bugInstance -> {
                    try {
                      return bugInstance
                          .getFullPath()
                          .contains(Utils.retrieveFullyQualifiedName(source));
                    } catch (ExceptionHandler exceptionHandler) {
                      exceptionHandler.printStackTrace();
                      return false;
                    }
                  }));
      print(fileOut);
    } catch (Exception e) {
      e.printStackTrace();
      assertNull(e);
    }
  }
}

In [0]:
main_SymCrypto_Example_File()

In [0]:
public void main_SymCrypto_Example_Package_File() {
  soot.G.v().reset();

  String source = symCrypto_Example_Package_File;
  String className = source;

  try {
    className = Utils.retrieveFullyQualifiedName(source);
  } catch (ExceptionHandler e) {
  }

  String fileOut = symCrypto_Example_Package_Json_File;

  new File(fileOut).delete();
  if (activate && isLinux) {
    String args =
        makeArg(argsIdentifier.FORMAT, EngineType.JAVAFILES)
            + makeArg(argsIdentifier.FORMATOUT, Listing.Default)
            + makeArg(argsIdentifier.SOURCE, source)
            + makeArg(argsIdentifier.JAVA, java7_home)
            + makeArg(argsIdentifier.NOEXIT)
            + makeArg(argsIdentifier.PRETTY)
            + makeArg(argsIdentifier.OUT, fileOut);

    try {
      EntryPoint.main(args.split(" "));
      Report report = Report.deserialize(new File(fileOut));

      assertFalse(report.getIssues().isEmpty());
      System.out.println(report.getIssues());

      String finalClassName = className.replaceAll("\\.", "/");

      assertTrue(
          report.getIssues().stream()
              .anyMatch(
                  bugInstance -> {
                    try {
                      return bugInstance
                          .getFullPath()
                          .contains(Utils.retrieveFullyQualifiedName(finalClassName));
                    } catch (ExceptionHandler exceptionHandler) {
                      exceptionHandler.printStackTrace();
                      return false;
                    }
                  }));
      print(fileOut);
    } catch (Exception e) {
      e.printStackTrace();
      assertNull(e);
    }
  }
}

In [0]:
main_SymCrypto_Example_Package_File()